<a href="https://colab.research.google.com/github/ytg000629/Dacon_Dobae/blob/main/modeling/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#라이브러리

import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalMaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# define constants
img_width, img_height = 299, 299
batch_size = 32
epochs = 20
num_classes = 19  # update this based on the number of classes you have

train_data_dir = "/content/drive/MyDrive/dacon/data/data_split/train"
val_data_dir = '/content/drive/MyDrive/dacon/data/data_split/val'
test_data_dir = '/content/drive/MyDrive/dacon/data/test'


In [ ]:
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# add a global spatial max pooling layer
x = base_model.output
x = GlobalMaxPooling2D()(x)

# add a fully-connected layer
x = Dense(1024, activation='relu')(x)

# and a logistic layer
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
!pip install tensorflow-addons==0.16.1

In [ ]:
import tensorflow_addons.metrics
w_f1 = tensorflow_addons.metrics.F1Score(num_classes=19, average = 'weighted')

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
from tensorflow.keras.losses import CategoricalCrossentropy
model.compile(optimizer=Adam(lr=0.001), 
              loss= CategoricalCrossentropy(label_smoothing=0.1),
              metrics=['accuracy', w_f1])

In [ ]:
# apply data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
# implement early stopping
callbacks = [
    ReduceLROnPlateau(monitor = 'val_f1_score', factor = 0.1, patience = 5, mode = 'min', verbose = 1),
    EarlyStopping(patience=10, verbose=1),
    ModelCheckpoint('model.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

# train the model on the new data for a few epochs
model.fit(
    train_generator,
    epochs=50,
    validation_data=val_generator,
    callbacks=callbacks)

In [ ]:
# Evaluate the model
# Load the best saved model based on validation data
model.load_weights('model.h5')

In [ ]:
# Prepare test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

In [ ]:
# Make predictions
predictions = model.predict(test_generator)

# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Create the classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)

print(report)

# Calculate F1 Score
f1score = f1_score(true_classes, predicted_classes, average='weighted')
print("F1 Score: ", f1score)

87910968/87910968 [==============================] - 3s 0us/step


Found 2200 images belonging to 19 classes.
Found 559 images belonging to 19 classes.
Epoch 1/20
69/69 [==============================] - ETA: 0s - loss: 4.8771 - accuracy: 0.4077 - auc: 0.8015 - precision: 0.4589 - recall: 0.3245 
Epoch 1: val_loss improved from inf to 1.49685, saving model to model.h5
69/69 [==============================] - 1281s 18s/step - loss: 4.8771 - accuracy: 0.4077 - auc: 0.8015 - precision: 0.4589 - recall: 0.3245 - val_loss: 1.4969 - val_accuracy: 0.5456 - val_auc: 0.9224 - val_precision: 0.6791 - val_recall: 0.4240
Epoch 2/20
69/69 [==============================] - ETA: 0s - loss: 1.5074 - accuracy: 0.5468 - auc: 0.9251 - precision: 0.6458 - recall: 0.4277
Epoch 2: val_loss did not improve from 1.49685
69/69 [==============================] - 90s 1s/step - loss: 1.5074 - accuracy: 0.5468 - auc: 0.9251 - precision: 0.6458 - recall: 0.4277 - val_loss: 1.5056 - val_accuracy: 0.5725 - val_auc: 0.9234 - val_precision: 0.7025 - val_recall: 0.4097
Epoch 3/20
69/6

Epoch 1/20
69/69 [==============================] - ETA: 0s - loss: 1.4149 - accuracy: 0.6114 - auc: 0.9370 - precision_1: 0.7243 - recall_1: 0.4764
Epoch 1: val_loss did not improve from 1.18633
69/69 [==============================] - 109s 1s/step - loss: 1.4149 - accuracy: 0.6114 - auc: 0.9370 - precision_1: 0.7243 - recall_1: 0.4764 - val_loss: 2.3052 - val_accuracy: 0.6351 - val_auc: 0.8992 - val_precision_1: 0.6465 - val_recall_1: 0.6118
Epoch 2/20
69/69 [==============================] - ETA: 0s - loss: 0.8335 - accuracy: 0.7386 - auc: 0.9753 - precision_1: 0.8346 - recall_1: 0.6605
Epoch 2: val_loss did not improve from 1.18633
69/69 [==============================] - 94s 1s/step - loss: 0.8335 - accuracy: 0.7386 - auc: 0.9753 - precision_1: 0.8346 - recall_1: 0.6605 - val_loss: 1.2608 - val_accuracy: 0.6816 - val_auc: 0.9488 - val_precision_1: 0.7657 - val_recall_1: 0.6315
Epoch 3/20
69/69 [==============================] - ETA: 0s - loss: 0.6384 - accuracy: 0.7895 - auc: 0.98

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import numpy as np
import os
from tensorflow.keras.preprocessing import image

real_test_data_dir = "/content/drive/MyDrive/open2/real_test"  # replace with your real test data directory

# load the best saved model
model.load_weights('model.h5')

# get all the image filenames
image_filenames = [f for f in os.listdir(real_test_data_dir) if os.path.isfile(os.path.join(real_test_data_dir, f))]

# sort the filenames
image_filenames.sort()

# for each image filename
for image_filename in image_filenames:
    # load the image
    img = image.load_img(os.path.join(real_test_data_dir, image_filename), target_size=(img_width, img_height))
    # convert image to array
    x = image.img_to_array(img)
    # scale the image
    x *= 1./255
    # add an extra dimension (for the batch)
    x = np.expand_dims(x, axis=0)
    # get the prediction from the model
    pred = model.predict(x)
    # get the most likely class
    pred_class = np.argmax(pred, axis=1)
    # print the predicted class
    print(f"The image {image_filename} is predicted to be in class {pred_class}")

1/1 [==============================] - 2s 2s/step
The image 000.png is predicted to be in class [7]
1/1 [==============================] - 0s 28ms/step
The image 001.png is predicted to be in class [10]
1/1 [==============================] - 0s 29ms/step
The image 002.png is predicted to be in class [18]
1/1 [==============================] - 0s 29ms/step
The image 003.png is predicted to be in class [7]
1/1 [==============================] - 0s 27ms/step
The image 004.png is predicted to be in class [10]
1/1 [==============================] - 0s 29ms/step
The image 005.png is predicted to be in class [18]
1/1 [==============================] - 0s 32ms/step
The image 006.png is predicted to be in class [10]
1/1 [==============================] - 0s 77ms/step
The image 007.png is predicted to be in class [7]
1/1 [==============================] - 0s 123ms/step
The image 008.png is predicted to be in class [10]
1/1 [==============================] - 0s 27ms/step
The image 009.png is pred

In [ ]:
import pandas as pd

# ... (the rest of your code)

# for each image filename
image_predictions = []
for image_filename in image_filenames:
    # load the image
    img = image.load_img(os.path.join(real_test_data_dir, image_filename), target_size=(img_width, img_height))
    # convert image to array
    x = image.img_to_array(img)
    # scale the image
    x *= 1./255
    # add an extra dimension (for the batch)
    x = np.expand_dims(x, axis=0)
    # get the prediction from the model
    pred = model.predict(x)
    # get the most likely class
    pred_class = np.argmax(pred, axis=1)
    # append the filename and its prediction to the list
    image_predictions.append((image_filename, pred_class[0]))

# convert the list to a DataFrame
df = pd.DataFrame(image_predictions, columns=['filename', 'predicted_class'])

# save the DataFrame to a csv file
df.to_csv('predictions.csv', index=False)


1/1 [==============================] - 0s 27ms/step


In [ ]:
from google.colab import files

files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Rename the columns
df = df.rename(columns={'filename': 'id', 'predicted_class': 'label'})

# Replace the values in the 'id' column
df['id'] = df['id'].str.replace('.png', '').str.zfill(3).apply(lambda x: f'TEST_{x}')

# Replace the values in the 'label' column
class_names = ['가구수정', '걸레받이수정', '곰팡이', '꼬임', '녹오염', '들뜸', '면불량', '몰딩수정', '반점', '석고수정', '오염', '오타공', '울음', '이음부불량', '창틀&문틀수정', '터짐', '틈새과다', '피스', '훼손']
class_dict = {i: class_name for i, class_name in enumerate(class_names, 1)}
df['label'] = df['label'].replace(class_dict)

# save the DataFrame to a csv file
df.to_csv('predictions.csv', index=False)

from google.colab import files

files.download('predictions.csv')

<ipython-input-8-964391593be4>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['id'] = df['id'].str.replace('.png', '').str.zfill(3).apply(lambda x: f'TEST_{x}')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>